In [3]:
import numpy as np
import pandas as pd
import glob
import math
import random
import os
import torch
from torchvision import datasets, transforms, models
from torch import nn, optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
# Images
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import geffnet

In [4]:
random.seed(41)
#Data Tranforms (Augmentation and Normalization)
train_transforms = transforms.Compose([
                                       # Rotación aleatoria de la imagen entre -30 y +30 grados para mejor entrenamiento
                                       transforms.RandomRotation(30),
                                       # Aplica una perspectiva aleatoria con una distorsión máxima del 40%
                                       transforms.RandomPerspective(distortion_scale=0.4),
                                       # Voltea la imagen horizontalmente con una probabilidad del 60%
                                       transforms.RandomHorizontalFlip(0.6),
                                       # Cambia el tamaño de la imagen a 224x224 píxeles
                                       transforms.Resize(size=(224,224)),
                                       # Cambia el tamaño de la imagen a 224x224 píxeles
                                       transforms.ToTensor(),
                                       # Normaliza cada canal de la imagen.
                                       # Los valores son estándares de ImageNet.
                                       # Resta la media y divide por la desviación estándar, canal por canal.
                                       transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([
                                       transforms.RandomRotation(30),
                                       transforms.RandomHorizontalFlip(0.6),
                                       transforms.Resize(size=(224,224)),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(size=(224,224)),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])])

In [ ]:

data_dir = ''
#Getting all the data with PyTorch Datasets
train_data = datasets.ImageFolder(data_dir + r'/train', transform= train_transforms)
val_data = datasets.ImageFolder(data_dir + r'/val', transform= valid_transforms)
test_data = datasets.ImageFolder(data_dir + r'/test', transform= test_transforms)

#Loading the data into PyTorch DataLoader
train_loader = torch.utils.data.DataLoader(train_data, batch_size= 64, shuffle = True,num_workers=4)
valid_loader = torch.utils.data.DataLoader(val_data, batch_size= 64, shuffle = True,num_workers=4)
test_loader = torch.utils.data.DataLoader(test_data, batch_size= 64, shuffle = True,num_workers=4)

#Creating a dictionary of all classes
classes = dict(zip(list(range(len(train_data.classes))),train_data.classes))